In [1]:
import requests as rq
import pandas as pd
import re
import numpy as np
from bs4 import BeautifulSoup as bs

In [2]:
apikey = "it's a secret"

In [3]:
payload = {}
headers = {'apikey': apikey}

base_url = "https://api.apilayer.com/spotify/"
endpoint_url = "album_tracks"
album_url = "album_metadata"

In [4]:
songs = pd.DataFrame(columns=['Title','Album','Track Number','Play Count','Duration in Seconds'])

In [5]:
ids = ['7mzrIsaAjnXihW3InKjlC3','7vzYp7FrKnTRoktBYsx9SF','2dqn5yOQWdyGwOpOIi9O4x','4hDok0OAJd57SGIT8xuWJH',
       '5EpMjweRD573ASl7uNiHym','1KlU96Hw9nlvqpBPlSqcTV','6kZ42qRrzov54LcAk4onW9','34OkZVpuzBa9y40DCy0LPR',
       '6DEjYFkNZh67HP7R9PSZvv','1NAmidJlEaVgA3MpcPFYGq','1pzvBxYgT6OVwJLtHkrdQK','6AORtDjduMM3bupSWzbTSG',
       '3lS1y25WAhcqJDATJK70Mq']

In [6]:
for j in range(len(ids)):
    params = {'id': ids[j]}
    r = rq.get(base_url + endpoint_url, params=params, headers=headers, data=payload)
    data = pd.DataFrame(r.json())
    data = data["data"].apply(pd.Series)
    data = data["tracks"].apply(pd.Series)
    data = data["items"].apply(pd.Series)
    table = pd.DataFrame(columns=['Title','Album','Track Number','Play Count','Duration in Seconds'])
    for i in range(data.shape[1]):
        track = data.iloc[0,i]
        table.loc[i,'Title'] = track['track']['name']
        table.loc[i,'Track Number'] = track['track']['trackNumber']
        table.loc[i,'Play Count'] = track['track']['playcount']
        table.loc[i,'Duration in Seconds'] = track['track']['duration']['totalMilliseconds']/1000
    ra = rq.get(base_url + album_url, params=params, headers=headers, data=payload)
    data = pd.DataFrame(ra.json())
    table['Album'] = data['data']['album']['name']
    songs = pd.concat([songs,table])

In [7]:
sids = ['45nqVXRAW0xv0wpU9JljPN','1O3BsjGx9plSOJ036ZY4Fl','1MPAXuTVL2Ej5x0JHiSPq8', '5JnnrTE35MQagEnEkFthiF']
num = [13, 10, 9, 0]

In [8]:
for j in range(len(sids)):
    params = {'id': sids[j]}
    r = rq.get(base_url + endpoint_url, params=params, headers=headers, data=payload)
    data = pd.DataFrame(r.json())
    data = data["data"].apply(pd.Series)
    data = data["tracks"].apply(pd.Series)
    data = data["items"].apply(pd.Series)
    table = pd.DataFrame(columns=['Title','Album','Track Number','Play Count','Duration in Seconds'])
    track = data.iloc[0,num[j]]
    table.loc[i,'Title'] = track['track']['name']
    table.loc[i,'Track Number'] = 1
    table.loc[i,'Play Count'] = track['track']['playcount']
    table.loc[i,'Duration in Seconds'] = track['track']['duration']['totalMilliseconds']/1000
    table['Album'] = 'Single'
    songs = pd.concat([songs,table])

In [9]:
songs['Track Number']=songs['Track Number'].apply(lambda x: pd.to_numeric(x))
songs['Play Count']=songs['Play Count'].apply(lambda x: pd.to_numeric(x))
songs['Duration in Seconds']=songs['Duration in Seconds'].apply(lambda x: pd.to_numeric(x))

In [10]:
songs['Album'] = songs['Album'].str.split(r'\ \(',expand=True)[0]
songs['Title'] = songs['Title'].str.split(r'\ \(',expand=True)[0]
songs['Title'] = songs['Title'].str.split(r'\ -',expand=True)[0]
songs['Album'] = songs['Album'].str.strip()
songs['Title'] = songs['Title'].str.strip()

In [11]:
songs_2 = songs.drop_duplicates(subset=['Title'], keep='first')
songs_2 = songs_2.drop(['Play Count','Duration in Seconds'], axis=1)
songs_play = songs.groupby([songs['Title']]).sum()
songs_play = songs_play.drop(['Duration in Seconds','Track Number'], axis=1)
songs_2 = songs_2.merge(songs_play, on=['Title'], how='left')
songs_2 = songs_2.reset_index()
songs_2 = songs_2.drop('index', axis=1)

In [12]:
songs_duration = songs.groupby([songs['Title']]).mean()
songs_duration = songs_duration.drop(['Play Count','Track Number'], axis=1)
songs_duration = songs_duration.reset_index()
songs_2 = songs_2.merge(songs_duration, on=['Title'], how='left')

In [13]:
url = "https://www.billboard.com/artist/taylor-swift/chart-history/hsi/"

In [14]:
r = rq.get(url)
soup = bs(r.text)

In [15]:
table = soup.find('div',{'class':'artist-chart-history // lrv-a-wrapper lrv-u-padding-lr-00@mobile-max u-padding-b-350@tablet lrv-u-padding-b-1@mobile-max'})

table_title = table.find_all('div')[0]
table_pos = table.find_all('div', {'o-chart-results-list__item // u-width-72 u-width-55@mobile-max lrv-u-flex lrv-u-flex-direction-column lrv-u-align-items-center lrv-u-justify-content-center u-background-color-white-064@mobile-max lrv-u-border-b-1 u-border-b-0@mobile-max lrv-u-border-color-grey-light'})
table_weeks = table.find_all('div', {'class':'o-chart-results-list__item // u-width-72 u-width-55@mobile-max lrv-u-flex lrv-u-align-items-center lrv-u-justify-content-center u-background-color-white-064@mobile-max lrv-u-border-b-1 u-border-b-0@mobile-max lrv-u-border-color-grey-light'})

In [16]:
songs = pd.DataFrame(columns=['Title','Peak Chart Position','Weeks on Chart'])

In [17]:
for i in range(189):
    songs.loc[i,'Title'] = table_title.find_all('h3')[i].get_text().replace('\n','').replace('\t','')
songs = songs.drop(0)
songs = songs.reset_index()
songs = songs.drop('index', axis=1)

In [18]:
for i in range(188):
    songs.loc[i,'Peak Chart Position'] = table_pos[i].get_text().replace('\n','').split('\t')[2]

In [19]:
for i in range(188):
    songs.loc[i,'Weeks on Chart'] = table_weeks[i].get_text().replace('\n','').replace('\t','')

In [20]:
songs['Peak Chart Position'] = pd.to_numeric(songs['Peak Chart Position'])
songs['Weeks on Chart'] = pd.to_numeric(songs['Weeks on Chart'])

In [21]:
songs['Title'] = songs['Title'].str.split(r'\ \(',expand=True)[0]
songs_new = pd.DataFrame(columns=['Title'])
songs_new['Title'] = songs.drop_duplicates(subset=['Title'])['Title']

In [22]:
peak = songs.groupby('Title').min()
songs_peak = songs_new.merge(peak, on='Title', how='left')
songs_peak = songs_peak.drop('Weeks on Chart', axis=1)

In [23]:
weeks = songs.groupby('Title').sum()
weeks = weeks.drop('Peak Chart Position', axis=1)
songs_chart = songs_peak.merge(weeks, on='Title', how='inner')

In [24]:
songs_final = songs_2.merge(songs_chart, on='Title', how='left')

In [25]:
songs_final['Peak Chart Position'] = songs_final['Peak Chart Position'].replace(np.nan, -1).apply(lambda x: int(x))
songs_final['Weeks on Chart'] = songs_final['Weeks on Chart'].replace(np.nan, -1).apply(lambda x: int(x))

In [26]:
songs_final

,Title,Album,Track Number,Play Count,Duration in Seconds,Peak Chart Position,Weeks on Chart
0,Tim McGraw,Taylor Swift,1,85080608,232.106,40,20
1,Picture To Burn,Taylor Swift,2,113459216,173.066,28,20
2,Teardrops On My Guitar,Taylor Swift,3,155880670,203.040,13,48
3,A Place in this World,Taylor Swift,4,24351942,199.200,-1,-1
4,Cold As You,Taylor Swift,5,25370648,239.013,-1,-1
...,...,...,...,...,...,...,...
195,Dear Reader,Midnights,20,62547002,225.194,45,2
196,Eyes Open,Single,1,46065828,244.586,19,13
197,Highway Don't Care,Single,1,184599299,276.880,22,20
198,I Don’t Wanna Live Forever,Single,1,1360488896,247.160,-1,-1


In [28]:
songs_final.to_csv('TaylorSwiftSongs.csv')